In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Câu 1

### VGG16

In [ ]:
data_path = "/kaggle/input/chest-xray-pneumonia"

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
Chest_XRay_IMG_SIZE = 227
channel = 3
Chest_XRay_BATCH_SIZE = 256
Chest_XRay_COLOR_MODE = 'rgb'

In [ ]:
import os

In [ ]:
chest_Xray_Path = '/kaggle/input/chest-xray-pneumonia/chest_xray'

In [ ]:
Chest_XRay_CLASSNAMES = {'NORMAL', 'PNEUMONIA'}

In [ ]:
chest_Xray_train_set = image_dataset_from_directory(
    chest_Xray_Path + '/train',
    labels = 'inferred',
    label_mode = 'binary',
    class_names = Chest_XRay_CLASSNAMES,
    color_mode = Chest_XRay_COLOR_MODE,
    batch_size = Chest_XRay_BATCH_SIZE,
    image_size = (Chest_XRay_IMG_SIZE, Chest_XRay_IMG_SIZE),
    interpolation = 'bilinear'
)

In [ ]:
chest_Xray_test_set = image_dataset_from_directory(
    chest_Xray_Path + '/test',
    labels = 'inferred',
    label_mode = 'binary',
    class_names = Chest_XRay_CLASSNAMES,
    color_mode = Chest_XRay_COLOR_MODE,
    batch_size = Chest_XRay_BATCH_SIZE,
    image_size = (Chest_XRay_IMG_SIZE, Chest_XRay_IMG_SIZE),
    interpolation = 'bilinear'
)

In [ ]:
chest_Xray_dev_set = image_dataset_from_directory(
    chest_Xray_Path + '/val',
    labels = 'inferred',
    label_mode = 'binary',
    class_names = Chest_XRay_CLASSNAMES,
    color_mode = Chest_XRay_COLOR_MODE,
    batch_size = Chest_XRay_BATCH_SIZE,
    image_size = (Chest_XRay_IMG_SIZE, Chest_XRay_IMG_SIZE),
    interpolation = 'bilinear'
)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize = (25,10))
for images, labels in chest_Xray_train_set.take(1):
    for i in range(10):
        plt.subplot(2 ,5, i+ 1)
        plt.imshow(np.squeeze(images[i].numpy().astype('uint8')))
        plt.axis('off')

In [ ]:
from keras.applications.vgg16 import VGG16
vgg = VGG16(include_top = False, input_shape=(227,227,3))

In [ ]:
# dong bang cac lop truoc do
for layer in vgg.layers:
    layer.trainable = False

In [ ]:
# tinh chinh dau ra
from keras.layers import Flatten, Dense, Input
from keras.models import Model

flat = Flatten()(vgg.layers[-1].output)
fc1 = Dense(1024, activation = 'relu')(flat)
output = Dense(1, activation = 'sigmoid')(fc1)

model = Model(inputs = vgg.inputs, outputs = output)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
model.fit(chest_Xray_train_set, epochs = 1, validation_data = chest_Xray_dev_set)

In [ ]:
y_pred_total = []
y_true = []
for img, label in chest_Xray_test_set:
    y_pred = model.predict(img)
    y_pred_total += [1 if i>0.5 else 0 for i in y_pred]
    y_true += np.array(label).flatten().tolist()

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
accuracy_score(y_true, y_pred_total)*100

In [ ]:
precision_score(y_true, y_pred_total)*100

In [ ]:
recall_score(y_true, y_pred_total)*100

In [ ]:
f1_score(y_true, y_pred_total)*100

 ### VGG-19

In [ ]:
from keras.applications.vgg19 import VGG19
vgg19 = VGG19(include_top = False, input_shape=(227,227,3))

In [ ]:
for layer in vgg19.layers:
    layer.trainable = False

In [ ]:
from keras.layers import Dense, Flatten, Input
from keras.models import Model
x = Flatten()(vgg19.output)
x = Dense(1024, activation = 'relu')(x)
output = Dense(1, activation = 'sigmoid')(x)
modelvgg19 = Model(inputs = vgg19.input, outputs = output)

In [ ]:
modelvgg19.summary()

In [ ]:
modelvgg19.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
modelvgg19.fit(chest_Xray_train_set, epochs = 1, validation_data = chest_Xray_dev_set)

In [ ]:
y_pred_total = []
y_true = []
for img, label in chest_Xray_test_set:
    y_pred = modelvgg19.predict(img)
    y_pred_total += [1 if i>0.5 else 0 for i in y_pred]
    y_true += np.array(label).flatten().tolist()

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print(f"accurancy score: {accuracy_score(y_true, y_pred_total)*100}")
print(f"precision score: {precision_score(y_true, y_pred_total)*100}")
print(f"recall score: {recall_score(y_true, y_pred_total)*100}")
print(f"f1 score: {f1_score(y_true, y_pred_total)*100}")

### ResNet 50

In [ ]:
from tensorflow.keras.applications import ResNet50V2
resnet50 = ResNet50V2(weights = "imagenet", input_shape = (227,227,3), include_top = False)

In [ ]:
for layer in resnet50.layers:
    layer.trainable = False

In [ ]:
from keras.layers import Dense, Flatten, Input
from keras.models import Model
x = Flatten()(resnet50.output)
x  = Dense(1024, activation = 'relu')(x)
output = Dense(1, activation = 'sigmoid')(x)
model_resnet50 = Model(inputs = resnet50.input, outputs = output)

In [ ]:
model_resnet50.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
model_resnet50.fit(chest_Xray_train_set, epochs=1, validation_data=chest_Xray_dev_set)

In [ ]:
y_pred_total = []
y_true = []
for img, label in chest_Xray_test_set:
    y_pred = model_resnet50.predict(img)
    y_pred_total += [1 if i>0.5 else 0 for i in y_pred]
    y_true += np.array(label).flatten().tolist()

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print(f"accurancy score: {accuracy_score(y_true, y_pred_total)*100}")
print(f"precision score: {precision_score(y_true, y_pred_total)*100}")
print(f"recall score: {recall_score(y_true, y_pred_total)*100}")
print(f"f1 score: {f1_score(y_true, y_pred_total)*100}")

## So sánh và kết luận

## Câu 2

In [1]:
!pip install Pillow
from PIL import Image

In [2]:
import torch
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms, models
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torchvision.datasets import ImageFolder

In [3]:
!git clone https://github.com/princesegzy01/Jewellery-Classification.git

Cloning into 'Jewellery-Classification'...
remote: Enumerating objects: 1831, done.
remote: Total 1831 (delta 0), reused 0 (delta 0), pack-reused 1831
Receiving objects: 100% (1831/1831), 49.51 MiB | 6.80 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [4]:
transform = transforms.Compose([transforms.Resize((224, 224)),transforms.ToTensor()])

In [5]:
data_train = ImageFolder(root='/content/Jewellery-Classification/dataset/training', transform=transform)
data_test = ImageFolder(root='/content/Jewellery-Classification/dataset/test', transform=transform)

In [6]:
train_dataloader = DataLoader(data_train, batch_size=8, shuffle=True, num_workers = 4)
test_dataloader = DataLoader(data_test, batch_size=8, shuffle=False, num_workers = 4)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


### VGG16

In [10]:
vgg16_weights = models.VGG16_Weights.DEFAULT
modelvgg16 = models.vgg16(weights = vgg16_weights)

In [11]:
for param in modelvgg16.parameters():
    param.requires_grad = False

In [12]:
modelvgg16.classifier[6] = nn.Linear(4096, 5)

In [15]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [16]:
modelvgg16.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [18]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(modelvgg16.parameters(), lr=0.001, momentum=0.9)

In [ ]:
next(iter(train_dataloader))

In [42]:
for epoch in range(10):
    modelvgg16.train()
    running_loss = 0.0
    total = 0.0
    correct_predictions = 0
    for inputs, labels in train_dataloader:
        optimizer.zero_grad()
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = modelvgg16(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        correct_predictions += (outputs.argmax(dim = 1) == labels).sum().item()
        total += len(labels)
    print(f"Epoch {epoch+1}/{10}, Loss: {running_loss/len(train_dataloader)}, accuracy{correct_predictions/total}")

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 1/2, Loss: 0.09664328496951291, accuracy0.9629629629629629


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 2/2, Loss: 0.06946933483399037, accuracy0.9763729246487867


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


...

Epoch 8/2, Loss: 0.07580162361777258, accuracy0.9757343550446999


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 9/2, Loss: 0.0670475174385724, accuracy0.9719029374201787


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 10/2, Loss: 0.07181076836007248, accuracy0.9706257982120051


In [56]:
modelvgg16.eval()
correct_predictions = 0
total = 0
label_predicted = []
label_true = []
with torch.no_grad():
  for img, label in test_dataloader:
    img = img.to(device)
    label = label.to(device)
    output = modelvgg16(img)
    label_predicted.append(output.argmax(dim = 1))
    label_true.append(label)
    correct_predictions += (output.argmax(dim = 1) == label).sum().item()
    total += len(label)
  print("accuracy: ", correct_predictions/total *100)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  


accuracy:  94.8


## VGG19

In [61]:
vgg19_weights = models.VGG19_Weights.DEFAULT
modelvgg19 = models.vgg19(weights = vgg19_weights)

for param in modelvgg19.parameters():
    param.requires_grad = False

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:05<00:00, 107MB/s]


In [64]:
modelvgg19.classifier[6] = nn.Linear(4096, 5)

In [62]:
modelvgg19.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [68]:
for epoch in range(10):
    modelvgg19.train()
    running_loss = 0.0
    total = 0.0
    correct_predictions = 0
    for inputs, labels in train_dataloader:
        optimizer.zero_grad()
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = modelvgg19(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        correct_predictions += (outputs.argmax(dim = 1) == labels).sum().item()
        total += len(labels)
    print(f"Epoch {epoch+1}/{10}, Loss: {running_loss/len(train_dataloader)}, accuracy{correct_predictions/total * 100}")

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 1/10, Loss: 1.61729910787271, accuracy27.84163473818646


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 2/10, Loss: 1.6145918150337375, accuracy26.81992337164751


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


...

Epoch 8/10, Loss: 1.6104474067687988, accuracy25.67049808429119


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 9/10, Loss: 1.621175888241554, accuracy26.245210727969347


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 10/10, Loss: 1.6134441470613285, accuracy25.67049808429119


In [69]:
modelvgg19.eval()
correct_predictions = 0
total = 0
label_predicted = []
label_true = []
with torch.no_grad():
  for img, label in test_dataloader:
    img = img.to(device)
    label = label.to(device)
    output = modelvgg19(img)
    label_predicted.append(output.argmax(dim = 1))
    label_true.append(label)
    correct_predictions += (output.argmax(dim = 1) == label).sum().item()
    total += len(label)
  print("accuracy: ", correct_predictions/total *100)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  


accuracy:  35.6


## ResNet50

In [71]:
resnet50_weights = models.ResNet50_Weights.DEFAULT
modelReNet50 = models.resnet50(weights = resnet50_weights)

In [72]:
for param in modelReNet50.parameters():
    param.requires_grad = False

In [75]:
modelReNet50.fc = nn.Linear(2048,5)

In [76]:
modelReNet50.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [77]:
for epoch in range(10):
    modelReNet50.train()
    running_loss = 0.0
    total = 0.0
    correct_predictions = 0
    for inputs, labels in train_dataloader:
        optimizer.zero_grad()
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = modelReNet50(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        correct_predictions += (outputs.argmax(dim = 1) == labels).sum().item()
        total += len(labels)
    print(f"Epoch {epoch+1}/{10}, Loss: {running_loss/len(train_dataloader)}, accuracy{correct_predictions/total * 100}")

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-pac

Epoch 1/10, Loss: 1.6133123411207784, accuracy14.240102171136654


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 2/10, Loss: 1.6135922281109556, accuracy14.75095785440613


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


...

Epoch 8/10, Loss: 1.6152620431111784, accuracy15.32567049808429


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 9/10, Loss: 1.6125297588961465, accuracy14.942528735632186


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 10/10, Loss: 1.6127674974957291, accuracy15.32567049808429


In [78]:
modelReNet50.eval()
correct_predictions = 0
total = 0
label_predicted = []
label_true = []
with torch.no_grad():
  for img, label in test_dataloader:
    img = img.to(device)
    label = label.to(device)
    output = modelReNet50(img)
    label_predicted.append(output.argmax(dim = 1))
    label_true.append(label)
    correct_predictions += (output.argmax(dim = 1) == label).sum().item()
    total += len(label)
  print("accuracy: ", correct_predictions/total *100)

accuracy:  18.4
